# Errors and Exceptions

# 错误和异常

No matter your skill as a programmer, you will eventually make a coding mistake.
Such mistakes come in three basic flavors:

- *Syntax errors:* Errors where the code is not valid Python (generally easy to fix)
- *Runtime errors:* Errors where syntactically valid code fails to execute, perhaps due to invalid user input (sometimes easy to fix)
- *Semantic errors:* Errors in logic: code executes without a problem, but the result is not what you expect (often very difficult to track-down and fix)

无论你编程的技巧有多高，你最终都会遇到代码的错误。这些错误有3个基本来源：

- *语法错误：* 代码不是合法的Python脚本（通常比较容易修复）
- *运行期错误：* 语法正确的代码在执行过程中发生错误，可能是用户不正确的输入造成的（有时比较容易修复）
- *逻辑错误：* 代码执行没有问题，但是结果并不如你所愿（通常难以跟踪和修复）

Here we're going to focus on how to deal cleanly with *runtime errors*.
As we'll see, Python handles runtime errors via its *exception handling* framework.

下面我们会聚焦于如何干净的处理*运行期错误*.正如我们将要看到的，Python通过*异常处理*框架处理运行期错误。

## Runtime Errors

## 运行期错误

If you've done any coding in Python, you've likely come across runtime errors.
They can happen in a lot of ways.

如果你已经编写了一些Python的代码了，你很可能已经遇到过运行期错误了。

For example, if you try to reference an undefined variable:

例如，如果你试图引用一个未定义的变量：

In [1]:
print(Q)

NameError: name 'Q' is not defined

Or if you try an operation that's not defined:

或者如果你尝试一个未定义的操作时：

In [2]:
1 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Or you might be trying to compute a mathematically ill-defined result:

又或者你尝试进行一个数学上错误定义的运算时：

In [3]:
2 / 0

ZeroDivisionError: division by zero

Or maybe you're trying to access a sequence element that doesn't exist:

又或者是你尝试访问一个序列中不存在的元素时：

（译者：这也是Python作为类型安全语言的标志之一，内存访问受到严格管理，不存在未定义的行为）

In [4]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range

Note that in each case, Python is kind enough to not simply indicate that an error happened, but to spit out a *meaningful* exception that includes information about what exactly went wrong, along with the exact line of code where the error happened.
Having access to meaningful errors like this is immensely useful when trying to trace the root of problems in your code.

你应该已经注意到了，在上述错误发生时，Python不是仅仅指出发生了错误，而是提供了一个*有意义*的异常，包含了出错的具体信息和出错的那行代码的信息。获得这些有意义的错误信息能帮助你更容易定位到你代码的问题。

## Catching Exceptions: ``try`` and ``except``

## 捕捉异常：`try` 和 `except`

The main tool Python gives you for handling runtime exceptions is the ``try``...``except`` clause.
Its basic structure is this:

Python用来处理运行期异常的最主要工具是`try...expect`语法。最基本的结构如下：

In [5]:
try:
    print("this gets executed first")
except:  # try结构快中执行发生错误时，会执行except结构块的代码
    print("this gets executed only if there is an error")

this gets executed first


Note that the second block here did not get executed: this is because the first block did not return an error.
Let's put a problematic statement in the ``try`` block and see what happens:

你已经注意到了，`except`代码块并没有得到执行：这是因为在`try`代码块中并没有发生错误。现在让我们放入一行有问题的语句在`try`代码块中，然后看看结果：

In [6]:
try:
    print("let's try something:")
    x = 1 / 0 # ZeroDivisionError
except:
    print("something bad happened!")

let's try something:
something bad happened!


Here we see that when the error was raised in the ``try`` statement (in this case, a ``ZeroDivisionError``), the error was caught, and the ``except`` statement was executed.

One way this is often used is to check user input within a function or another piece of code.
For example, we might wish to have a function that catches zero-division and returns some other value, perhaps a suitably large number like $10^{100}$:

这里我们可以看到当`try`代码块中发生了一个错误时（在本例中，是一个`ZeroDivisionError`错误），那么这个错误会被捕捉，然后`except`代码块就会得到执行。

这种方式经常用于在函数或其他代码块中检查用户的输入。例如，当我们的函数体中发生被0除时，我们可以返回另外一个数，比方说一个非常巨大的数值$10^{100}$：

In [7]:
def safe_divide(a, b):  # 定义一个安全除法函数
    try:
        return a / b  # 无错误发生，返回a/b的商
    except:  # 发生被0除错误时，返回10的100次方
        return 1E100

In [8]:
safe_divide(1, 2)

0.5

In [9]:
safe_divide(2, 0)

1e+100

There is a subtle problem with this code, though: what happens when another type of exception comes up? For example, this is probably not what we intended:

上面的函数代码有个小问题，如果`try`代码块执行过程中还有另外的异常发生，会发生什么？例如，下面的结果可能并非你的初衷：

In [10]:
safe_divide (1, '2')

1e+100

Dividing an integer and a string raises a ``TypeError``, which our over-zealous code caught and assumed was a ``ZeroDivisionError``!
For this reason, it's nearly always a better idea to catch exceptions *explicitly*:

使用一个字符串去除一个整数会产生一个`TypeError`，然而我们上面的代码并未区分，因此我们还是按照`ZeroDivisionError`的逻辑进行了处理。因此，总是去捕捉一个*更特定的*异常永远是一个更好的方法：

In [11]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [12]:
safe_divide(1, 0)

1e+100

In [13]:
safe_divide(1, '2')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

We're now catching zero-division errors only, and letting all other errors pass through un-modified.

现在我们仅仅捕捉了0除错误，其他的错误将不会走到`except`中。

## Raising Exceptions: ``raise``

## 产生异常：`raise`

We've seen how valuable it is to have informative exceptions when using parts of the Python language.
It's equally valuable to make use of informative exceptions within the code you write, so that users of your code (foremost yourself!) can figure out what caused their errors.

我们已经看到产生的异常详细信息对于我们定位和修复问题来说是多么重要。同样的，对于你自己写的代码来说，产生异常的详细信息也是非常有价值的，这样才能方便使用你代码的用户（首先是你自己！）能发现哪里出了问题。

The way you raise your own exceptions is with the ``raise`` statement. For example:

使用`raise`关键字来产生你自己的异常，例如：

In [14]:
raise RuntimeError("my error message")

RuntimeError: my error message

As an example of where this might be useful, let's return to our ``fibonacci`` function that we defined previously:

我们希望能有一个更有实际意义的例子来进行说明，让我们回头看我们的`fibonacci`函数：

In [15]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

One potential problem here is that the input value could be negative.
This will not currently cause any error in our function, but we might want to let the user know that a negative ``N`` is not supported.
Errors stemming from invalid parameter values, by convention, lead to a ``ValueError`` being raised:

函数中一个可能的问题是输入的参数可能为负数。在这种情况下，我们上面的函数不会产生任何错误，但是我们可能希望让我们代码的用户知道负数的`N`是不被接受的。对于错误的输入参数来说，习惯性将错误定义为`ValueError`：

In [16]:
def fibonacci(N):
    if N < 0:
        raise ValueError("N must be non-negative")  # N为负数是，抛出一个ValueError异常
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [17]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [18]:
fibonacci(-10)

ValueError: N must be non-negative

Now the user knows exactly why the input is invalid, and could even use a ``try``...``except`` block to handle it!

现在用户就可以知道为什么这个输入参数是错误的，还可以使用`try...except`代码块来处理这个错误。

In [19]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))  # 这里调用fibonacci函数时，会抛出一个ValueError，然后被下面的except捕捉到
except ValueError:
    print("Bad value: need to do something else")

trying this...
Bad value: need to do something else


## Diving Deeper into Exceptions

## 深入了解异常

Briefly, I want to mention here some other concepts you might run into.
I'll not go into detail on these concepts and how and why to use them, but instead simply show you the syntax so you can explore more on your own.

下面我们来简略的说明一些与异常相关的概念。这里不会详细地说明这些概念以及如何和为什么使用他们，只会简单的列出使用他们的语法，让读者自己去探索研究。

### Accessing the error message

### 获得错误消息

Sometimes in a ``try``...``except`` statement, you would like to be able to work with the error message itself.
This can be done with the ``as`` keyword:

有的情况下，除了捕捉异常，你还需要对错误消息本身进行操作。使用`as`语法可以完成这项工作：

In [20]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is:  ", type(err))  # 错误的类型
    print("Error message is:", err)  # 错误的消息

Error class is:   <class 'ZeroDivisionError'>
Error message is: division by zero


With this pattern, you can further customize the exception handling of your function.

使用这种方法，你可以定制化对异常的处理方式。

### Defining custom exceptions

### 自定义异常

In addition to built-in exceptions, it is possible to define custom exceptions through *class inheritance*.
For instance, if you want a special kind of ``ValueError``, you can do this:

除了內建的异常，你可以采用*类继承*的方式自定义异常。例如，如果你需要使用一种特别的`ValueError`，你可以这样做：

In [21]:
class MySpecialError(ValueError):
    pass

raise MySpecialError("here's the message")

MySpecialError: here's the message

This would allow you to use a ``try``...``except`` block that only catches this type of error:

这可以允许你使用`try...except`结构块来捕捉这种自定义的异常：

In [22]:
try:
    print("do something")
    raise MySpecialError("[informative error message here]")
except MySpecialError:
    print("do something else")

do something
do something else


You might find this useful as you develop more customized code.

在你开发自己的项目时，这种自定义的异常会很有帮助。

## ``try``...``except``...``else``...``finally``

## `try...except...else...finally`

In addition to ``try`` and ``except``, you can use the ``else`` and ``finally`` keywords to further tune your code's handling of exceptions.
The basic structure is this:

除了使用`try`和`except`，你还可以使用`else`和`finally`关键字进一步优化对异常处理的代码。基本结构如下：

In [23]:
try:
    print("try something here")  # try结构块
except:
    print("this happens only if it fails")  # 异常发生时执行的结构块
else:
    print("this happens only if it succeeds") # 无异常发生时的结构块
finally:
    print("this happens no matter what")  # 任何情况下都会最后执行的结构块

try something here
this happens only if it succeeds
this happens no matter what


The utility of ``else`` here is clear, but what's the point of ``finally``?
Well, the ``finally`` clause really is executed *no matter what*: I usually see it used to do some sort of cleanup after an operation completes.

`else`的含义在这里是比较容易理解的，但`finally`是什么作用呢？`finally`结构包含着*无论什么情况*都会执行的代码，通常可以用来执行一些资源的释放和清理操作。